Data Mining and Visualisation 2019-2020 <br>
Practical 2 - K-means Clustering <br>
Teaching Assistant Muhammad Usman <br>

# INTRODUCTION <br>

In this exercise, you are expected to write a Python code to implement the k-means algorithm. You can write you own code to implement the algorithm detailed in the lecture; you can also use existing software packages to implement the algorithm.

# Hint <br>

Useful for Assessment 1

Some code has been provided already.
Solution will be uploaded as well.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
from sklearn.cluster import KMeans
from sklearn import metrics

from google.colab import files


ModuleNotFoundError: No module named 'google'

# Step 1
Download the file kmeans.csv from blackboard. There are two attributes in the data.

In [ ]:
def get_data():
    # complete - load the data
    data = pd.read_csv(io.StringIO(files.upload()['kmeans.csv'].decode('utf-8')))
  
    # complete - shuffle data rows
    return data

def shuffle_data(data):
    return data.sample(frac=1).reset_index(drop=True)


In [ ]:
data = get_data()
data.head()

# Step 2
Show the scatter plots of the data; you will see that there are three clusters.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
for i, dat in enumerate([data, shuffle_data(data)]):
    axes[i].scatter(dat.iloc[:, 0], dat.iloc[:, 1], color=plt.cm.jet(data.index))
    axes[i].grid(color='lightgrey')
plt.show()


In [ ]:
data = shuffle_data(data)
data.head()

# Steps 3
Write a Python code to implement the k-means algorithm on the data. You can use the Python module sklearn to import the k-means algorithm, for example, `from sklearn.cluster import KMeans’.

In [ ]:
def kmeans_clustering(data, K):
    # complete - define the model
    model = KMeans(n_clusters=K)

    # complete - fit the model to data
    model.fit(data)
    centroid = model.cluster_centers_
    labels = model.labels_
    
    distance = model.inertia_
    cluster_set = {}
    for i,x in enumerate(data):
        xlabel = labels[i]
        if xlabel not in cluster_set.keys():
            cluster_set[xlabel] = np.reshape(x,(1,len(x)))
        else:
            cluster_set[xlabel] = np.append(cluster_set[xlabel],np.reshape(x,(1,len(x))), axis=0)
    return centroid, distance, cluster_set


In [ ]:
K = 3
centroids, distance, clusters_set = kmeans_clustering(data.values, K)
distance

In [ ]:
clusters_set.keys()

# Step 4
Try to write a Python code to implement the k-means algorithm introduced in the Lecture. This is also needed for the assessment.


In [ ]:
def my_kmeans(data_, k):
    n_samples, n_features = data_.shape
    labels_set = set(range(k))

    # assign labels randomly
    if k > 1:
        labels_ = np.random.randint(low=0, high=k-1, size=n_samples)
    else:
        labels_ = np.zeros(n_samples)
    labels_old = labels_ + 1  # make sure it's different than labels_
    
    # initialise centroids
    centroids_ = np.zeros((k, n_features))

    it = 0

    while (labels_ != labels_old).sum():
        it += 1
        labels_old = labels_[:]

        # assign data points to clusters
        clusters_ = {label: data_[np.where(labels_==label)] for label in labels_set}

        # update centroids
        for i in labels_set:
            if clusters_[i].size:
                centroids_[i] = np.mean(clusters_[i], axis=0)
            else:
                # move the centroid to coincide with a random data point
                # - in the next iteration, at least one sample will belong to it
                centroids_[i] = data_[np.random.randint(0, n_samples)]

        # compute distances
        distances_ = metrics.pairwise.euclidean_distances(data_, centroids_)
        
        # get new labels
        labels_ = distances_.argmin(axis=1)
        
    print(f"Process finished in {it} iterations")
    return centroids_, sum(distances_[np.arange(n_samples), labels_]**2), clusters_


In [ ]:
my_centroids, my_distance, my_clusters_set = my_kmeans(data.values, 3)
print(centroids, '\n\n', my_centroids)
print('\n', distance, my_distance)

# Step 5
Visualise the scatter plot clusters with different colours and their centroids.

In [ ]:
plt.figure()
for i, key in enumerate(sorted(clusters_set.keys())):
    cluster = clusters_set[key]
    plt.scatter(cluster[:,0],cluster[:,1],alpha=0.5)#,cmap=matplotlib.colors.ListedColormap(colors[plt.scatter(centroid[key][0],centroid[key][1],alpha=1)#,cmap=matplotlib.colors.ListedColormap(# choose the number of clusters
    plt.plot(*tuple(centroids[i]), 'X', ms=15)

plt.show()


# Step 6
Optimal number of clusters: The number of clusters has to be provided before applying k-means algorithm to the data. However, we do not know the optimal number of clusters corresponding to this data. To find the optimal number of clusters, we can optimise the aggregate distance over all the data. Therefore, to find the optimal number, your task is to do the following:

  a) vary the number of clusters K from 1 to 10, and apply k-means to the data for each K;

  b) calculate the aggregate distance for each K;

  c) visualise the aggregate distance against K;

  d) select the optimal K where _mountain_ ends and _rubbles_ begins.


In [ ]:
def agg_distance(data, cluster_func=kmeans_clustering):
    distances_ = np.zeros((2, 10))
    for i in range(distances_.shape[1]):
        distances_[0, i] = i + 1
        distances_[1, i] = cluster_func(data.values, i+1)[1]
    return distances_

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
for i, func in enumerate([kmeans_clustering, my_kmeans]):
  all_distances = agg_distance(data, func)
  axes[i].plot(all_distances[0], all_distances[1], marker='o')
  axes[i].set_xlabel('number of clusters')
  axes[i].set_ylabel('aggregate distance')
  axes[i].grid()
